In [36]:
from youtube_transcript_api import YouTubeTranscriptApi, YouTubeTranscriptApiException     ##For Public videos only
import youtube_transcript_api

# Check what's available in the package
#dir(youtube_transcript_api)

In [37]:
#dir(YouTubeTranscriptApi)

In [ ]:
youtube_video_url1 = 'https://www.youtube.com/watch?v=d4yCWBGFCEs'
a1 = youtube_video_url1.split("/")
print('a1: ', a1)
b1 = youtube_video_url1.split("/")[-1]
print('b1: ', b1)
c1 = youtube_video_url1.split("/")[-1].split('?v=')
print('c1: ', c1)
d1 = youtube_video_url1.split("/")[-1].split('?v=')[1]
print('d1: ', d1)

a1:  <re.Match object; span=(30, 43), match='v=xbnHfRPFvyc'>
b1:  watch?v=d4yCWBGFCEs
c1:  ['watch', 'd4yCWBGFCEs']
d1:  d4yCWBGFCEs


In [8]:
youtube_video_url2 = 'https://www.youtube.com/watch?v=67_aMPDk2zw'
a2 = youtube_video_url2.split("/")
print('a2: ', a2)
b2 = youtube_video_url2.split("/")[-1]
print('b2: ', b2)
c2 = youtube_video_url2.split("/")[-1].split("?")
print('c2: ', c2)
d2 = youtube_video_url2.split("/")[-1].split("v=")[1]
print('d2: ', d2)

a2:  ['https:', '', 'www.youtube.com', 'watch?v=67_aMPDk2zw']
b2:  watch?v=67_aMPDk2zw
c2:  ['watch', 'v=67_aMPDk2zw']
d2:  67_aMPDk2zw


In [ ]:
# Try to get the manually created transcript first
youtube_video_url = 'https://www.youtube.com/watch?v=eMlx5fFNoYc'
video_id = youtube_video_url.split("v=")[1]
print(video_id)
transcript_list = YouTubeTranscriptApi().list(video_id)
#print(transcript_list)

languages = ['en', 'hi', 'ta', 'te', 'bn', 'gu', 'fr', 'de', 'es']
transcript_obj = transcript_list.find_manually_created_transcript(languages)
print(transcript_obj)

transcript_text = transcript_obj.fetch()
print(transcript_text)

transcript = ""
for item in transcript_text:
    transcript += " " + item.text 
print(transcript)    

eMlx5fFNoYc
en ("English")[TRANSLATABLE]
FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='In the last chapter, you and I started to step', start=0.0, duration=1.967), FetchedTranscriptSnippet(text='through the internal workings of a transformer.', start=1.967, duration=2.053), FetchedTranscriptSnippet(text='This is one of the key pieces of technology inside large language models,', start=4.56, duration=3.32), FetchedTranscriptSnippet(text='and a lot of other tools in the modern wave of AI.', start=7.88, duration=2.32), FetchedTranscriptSnippet(text='It first hit the scene in a now-famous 2017 paper called Attention is All You Need,', start=10.98, duration=4.54), FetchedTranscriptSnippet(text='and in this chapter you and I will dig into what this attention mechanism is,', start=15.52, duration=4.266), FetchedTranscriptSnippet(text='visualizing how it processes data.', start=19.786, duration=1.914), FetchedTranscriptSnippet(text="As a quick recap, here's the important context I

In [20]:
# If no manually created transcript is available, fallback to auto-generated
transcript_obj = transcript_list.find_generated_transcript(languages)
print(transcript_obj)

transcript_text = transcript_obj.fetch()
print(transcript_text)

transcript = ""
for item in transcript_text:
    transcript += " " + item.text
print(transcript)    

en ("English (auto-generated)")[TRANSLATABLE]
FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='in the last chapter you and I started to', start=0.04, duration=3.44), FetchedTranscriptSnippet(text='step through the internal workings of a', start=1.64, duration=3.6), FetchedTranscriptSnippet(text='transformer this is one of the key', start=3.48, duration=3.399), FetchedTranscriptSnippet(text='pieces of Technology inside large', start=5.24, duration=3.8), FetchedTranscriptSnippet(text='language models and a lot of other tools', start=6.879, duration=4.601), FetchedTranscriptSnippet(text='in the Modern Wave of AI it first hit', start=9.04, duration=4.719), FetchedTranscriptSnippet(text='the scene in a now famous 2017 paper', start=11.48, duration=4.399), FetchedTranscriptSnippet(text='called attention is all you need and in', start=13.759, duration=4.041), FetchedTranscriptSnippet(text='this chapter you and I will dig into', start=15.879, duration=3.921), FetchedTranscriptSnippet

In [44]:
from youtube_transcript_api import YouTubeTranscriptApi
import re

def extract_transcript_details(youtube_video_url):
    try:
        # Extract the video ID from the URL more robustly
        video_id_match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', youtube_video_url)
        if not video_id_match:
            raise Exception('Invalid YouTube URL')
        video_id = video_id_match.group(1)

        # List all available transcripts for the video
        transcript_list = YouTubeTranscriptApi().list(video_id)    

        # Try to get the manually created transcript first
        transcript = ""
        languages = ['en', 'hi', 'ta', 'te', 'bn', 'gu', 'fr', 'de', 'es']
        
        try:
            transcript_obj = transcript_list.find_manually_created_transcript(languages)
            transcript_text = transcript_obj.fetch()
            for item in transcript_text:
                transcript += " " + item.text
        
        # If no manually created transcript is available, fallback to auto-generated
        except Exception as e:
            if "NoTranscriptFound" in str(type(e).__name__):
                try:
                    transcript_obj = transcript_list.find_generated_transcript(languages)
                    transcript_text = transcript_obj.fetch()
                    for item in transcript_text:
                        transcript += " " + item.text
                except Exception as e2:
                    if "NoTranscriptFound" in str(type(e2).__name__):
                        # If no transcript in preferred languages, try any available transcript
                        available_transcripts = list(transcript_list)
                        if available_transcripts:
                            transcript_obj = available_transcripts[0]
                            transcript_text = transcript_obj.fetch()
                            for item in transcript_text:
                                transcript += " " + item.text
                        else:
                            raise Exception('No transcripts found for this video in any language.')
                    else:
                        raise e2
            else:
                raise e
        
        return transcript.strip()
    
    except Exception as e:
        if "TranscriptsDisabled" in str(type(e).__name__):
            raise Exception('Transcripts are disabled for this video.')
        elif "NoTranscriptFound" in str(type(e).__name__):
            raise Exception('No transcripts found for this video.')
        else:
            raise Exception(f'An error occurred: {str(e)}')

In [45]:
result = extract_transcript_details('https://www.youtube.com/watch?v=xbnHfRPFvyc')
print(result)

नमस्कार वेलकम टू स्टडी आ मैं हूं प्रशांत धवन और इस वीडियो में हम बात करने वाले हैं बहुत ही इंपोर्टेंट टॉपिक के बारे में हाल ही में पाकिस्तानी मीडिया ने हर जगह यह रिपोर्ट फैला दी कि पाकिस्तान में बहुत ही बड़े स्केल पर ऑयल और नेचुरल गैस के रिजर्व्स मिल चुके हैं नाउ इन्होंने यहां तक कह दिया यह जो आप लाइन देख रहे हैं यह सब पाकिस्तान मीडिया से उठाई गई है यहां पर लिखा है कि इतना ऑयल मिला है पाकिस्तान में जिससे पाकिस्तान की त तकदीर बदल सकती है इट कैन चेंज द डेस्टिनी ऑफ पाकिस्तान कई पाकिस्तानी न्यूज़ आउटलेट्स यह क्लेम कर रहे हैं कि दुनिया में चौथे सबसे बड़े ऑयल रिजर्व मिले हैं पाकिस्तान को अपने कोस्टल एरिया के पास नाउ मुझे यहां पर यह बोलना पड़ेगा एक देश जब और बाकी सेक्टर्स में फेल कर गया हो कोई आईटी इंडस्ट्री ना हो कोई मैन्युफैक्चरिंग ना हो कोई सर्विस इंडस्ट्री ना हो कुछ बेसिकली आपके कंट्री को इंटरनेशनली किसी चीज के लिए किसी चीज के एक्सपोर्ट के लिए जाना नहीं जाता सिर्फ लिटरली आपका कंट्री फेमस है टेररिज्म के लिए तो काफी बार आप कह लीजिए डिस्कवरी के कहीं से कुछ मिल जाए कहीं से ऑयल मिल जाए कहीं से

In [48]:
from langchain_groq import ChatGroq
import os

In [55]:
from dotenv import load_dotenv
load_dotenv()

True

In [65]:
llm = ChatGroq(model = 'llama-3.1-8b-instant',
               temperature=0,
               groq_api_key = os.getenv('GROQ_API_KEY')
               )


In [66]:
prompt = """You're an Youtube video summarizer. Take the transcript, summarize the video and also return important pointer""" 

In [67]:
def generate_content(transcript_text, prompt):
    model = llm
    response = model.invoke(prompt + transcript_text)
    return response

In [68]:
response = generate_content(transcript_text=result, prompt=prompt)
print(response.content)

वीडियो का सारांश:

प्रशांत धवन ने एक वीडियो में पाकिस्तान में ऑयल और नेचुरल गैस के बड़े स्केल पर रिजर्व्स मिलने की खबरों की जांच की है। उन्होंने कहा कि पाकिस्तानी मीडिया ने यह खबर फैलाई है कि पाकिस्तान में इतना ऑयल मिला है कि पाकिस्तान की तकदीर बदल सकती है, लेकिन उन्होंने कहा कि यह खबरें पुरानी हैं और पाकिस्तान में अभी तक कोई बड़ा ऑयल रिजर्व नहीं मिला है।

प्रशांत धवन ने कहा कि पाकिस्तान के पीएम इमरान खान ने 2018 में भी यही दावा किया था कि पाकिस्तान में ऑयल और नेचुरल गैस की बड़ी डिस्कवरी हुई है, लेकिन जब खुदाई हुई तो कुछ भी नहीं मिला। उन्होंने कहा कि पाकिस्तान के पीएम ने अब फिर से यही दावा किया है कि पाकिस्तान में 5 बिलियन डॉलर की इन्वेस्टमेंट हुई है और ऑयल और नेचुरल गैस की एक्सप्लोरेशन शुरू हो गई है।

प्रशांत धवन ने कहा कि यह सिर्फ एक इंटरनेशनल स्कैम है और पाकिस्तान के पीएम ने इन्वेस्टर्स को फंसाने की कोशिश कर रहे हैं। उन्होंने कहा कि अगर पाकिस्तान में ऑयल और नेचुरल गैस की एक्सप्लोरेशन होती है, तो इसका बड़ा हिस्सा सऊदी अरेबिया और यूएई के इन्वेस्टर्स को जाएगा।

वीडियो के अंत में, प्रशा